# Using Interact

In [1]:
import warnings
import numpy as np
from scipy import signal
from ipywidgets import *
import matplotlib.pyplot as plt
from IPython.display import Audio, display

In [2]:
octave_cents = 1200
halftone_cents = 100


def ratio2cents(ratio):
  return np.log2(ratio) * octave_cents


def cents2ratio(cents):
  return np.power(2, cents / octave_cents)


def octave_fit(cents):
  return cents % octave_cents


def tet_fit(cents):
  '''Fit to 12-TET half steps
  '''
  return round(cents / halftone_cents) * halftone_cents


fifth_cents = ratio2cents(3 / 2)  # Pythagorean fifth


def edo(num_notes):
  '''Equal division of the octave
  '''
  tone_step = octave_cents / num_notes
  note_cents = [(n * tone_step) % octave_cents for n in range(num_notes) ]
  return note_cents


def pythagorean_tuning(num_notes, ref_note=0):
  note_cents = [octave_fit(n * fifth_cents) for n in range(num_notes)]

  ref_cents = note_cents[ref_note]
  note_cents = [octave_fit(cents - ref_cents) for cents in note_cents]

  note_cents.sort()
  return note_cents


def equal_temperament(num_notes, ref_note=0):
  pyth = pythagorean_tuning(num_notes, ref_note=ref_note)
  note_cents = [tet_fit(p) for p in pyth]

  return note_cents



In [3]:
def time2samples(t, rate):
    """
    Convert time to number of samples at a certain rate.
    :param t: Time in seconds
    :param rate: Sampling rate.
    :return: Number of samples for the given time at the given rate.
    """
    return round(t * rate)


def db2mag(db):
    return 10 ** (db / 20)


def piecewiselin(x, y):
    """
    Create piecewise linear array
    :param x: Lengths of the segments in number of samples.
    :param y: Point values. Its length must be 1 greater than the lenght of x.
    :return: Piecewise linear array.
    """
    xlen = len(x)
    ylen = len(y)

    if (ylen - xlen) != 1:
        raise ValueError(f"Invalid dimensions len(x)={xlen}, len(y)={ylen}. len(x) must equal len(y) - 1.")

    y_start = y[:-1]
    y_stop = y[1:]

    pwl = np.array([])
    for start, stop, num in zip(y_start, y_stop, x):
        pwl = np.append(pwl, np.linspace(start, stop, num))

    return pwl

def adsr(adsr_durations, adsr_levels, total_duration=None, sustain_fade=1):
    """
    Generate Attack-Decay-Sustain-Release curve
    :param adsr_durations: Times of the diferent stages in number of samples. The order is (attack, decay, sustain, release).
    :param adsr_levels: dB levels of the ADSR curve, as (dB Floor, dB Peak, dB Sustain, dB Release)
    :param total_duration: Extrapolated duration beyond release. Default is None, meaning restricting to release time 
    :return: Adsr curve in linear magnitude
    """
    n_attack, n_decay, n_sustain, n_release = adsr_durations

    keystroke_duration = n_attack + n_decay + n_sustain + n_release
    
    db_floor, db_peak, db_sustain, db_release = adsr_levels

    if total_duration is not None:
      if total_duration < keystroke_duration:
        raise ValueError("Total duration must be greater or equal than the overall keystroke duration")
        
      fadeout = total_duration - keystroke_duration

      db_slope = (db_floor - db_sustain) / n_release
      db_release += db_slope * fadeout

      n_release += fadeout

    if max(adsr_levels) > -6.0:
        raise warnings.warn(f"Your dB levels are {adsr_levels}. Levels greater than -6 dB may saturate.", UserWarning)
    

    #################################################################
    #                                                               #
    #                       ADSR Curve                              #
    # dB Level ^                                                    #
    #          |                                                    #
    #    Peak  |        X                                           #
    #          |       X XX                                         #
    #          |      X    XX                                       #
    # Sustain  |     X       XXXXXXXXXXXXX                          #
    #          |    X                     XX                        #
    #          |   X                        XX                      #
    #          |  X                           XX                    #
    #          | X                              XX                  #
    #   Floor  |X                                 X                 #
    #          +--------------------------------------------> Time  #
    #          |-----|------------|---------|-------|-------|       #
    #           Attack  Decay      Sustain   Release Fadeout        #
    #################################################################

    adsr_curve = piecewiselin((n_attack, n_decay, n_sustain, n_release),
                              (db_floor, db_peak, db_sustain,
                               db_sustain - sustain_fade, db_release))
    magnitudes = db2mag(adsr_curve)

    return magnitudes


def lowpass_filter(x, fc=8000, order=6, fs=44100):
  """
    Apply Butterworth lowpass-filter to signal
    :param x: Input signal.
    :param fc: Cutoff frequency for the filtering. Default is 8 kHz.
    :param fs: Sampling frequency. Default is 44.1 kHz
    :param order: Order of the butterworth lowpass filter. Default is 4
    :return: Filtered signal
  """

  b, a = signal.butter(order, fc * 2 / fs)
  y = signal.lfilter(b, a, x)

  return y
  

def sawtooth_harmonics(f0, t, n_harmonics=6, power=1):

  wave = np.zeros_like(t)
  for n in np.arange(1, n_harmonics + 1):
    a = 1 / (n ** power)
    wave += a * np.sin(2 * np.pi * n * f0 * t)

  return wave


def synthesize(note_frequencies, fref=261.626, fc=1500, fs=44100, fadeout=1,
               attack=.1, decay=.1, sustain=-7, release=.9, sus_time=.1,
               floor=-80, peak=-6, rel_level=-20, spacing=.6):
  
  # TODO add diferent initial and final floor initial -60, final -30 or so

  attack_samples = time2samples(attack, fs)
  decay_samples = time2samples(decay, fs)
  sustain_samples = time2samples(sus_time, fs)
  release_samples = time2samples(release, fs)
  fadeout_samples = time2samples(fadeout, fs)

  sample_spacing = time2samples(spacing, fs)
  total_samples = sample_spacing * len(note_frequencies)  + fadeout_samples

  adsr_samples = (attack_samples, decay_samples,
                  sustain_samples, release_samples)
  adsr_levels = (floor, peak, sustain, rel_level)

  data = np.zeros(total_samples)

  for n, f_cent in enumerate(note_frequencies):

      t_offset = n * sample_spacing
      t_length = total_samples - t_offset
      t = np.arange(t_length) / fs

      f_hz = fref * np.power(2, f_cent / octave_cents)
      
      waveform = sawtooth_harmonics(f_hz, t, power=3)
      adsr_curve = adsr(adsr_samples, adsr_levels, total_duration=t_length)
      wave = lowpass_filter(waveform * adsr_curve, fc=fc)

      data[t_offset:] += wave

  return data

In [4]:
%matplotlib widget

n_notes = 7
fifths = np.mod(np.arange(n_notes) * fifth_cents, octave_cents)
fifth_order = np.argsort(fifths)

r = np.ones(n_notes + 1)

note_keys = np.array([f"N{n + 1}" for n in range(n_notes)])

fifth_keys = note_keys[fifth_order]
fifth_keys = np.append(fifth_keys, note_keys[0])

notes = equal_temperament(n_notes, ref_note=1)

edo_step = octave_cents / n_notes

# TODO return dict from functions

grid = GridspecLayout(n_notes + 1, 2)

note_default = {}
note_sliders = {}
slider_neighbours = {}
for n, k in enumerate(note_keys):

  min_val = 0 #  round(edo_step * n - edo_step / 2)
  max_val = 1200 #round(edo_step * n + edo_step / 2)
  val = round(notes[n])
  
  note_sliders[k] = IntSlider(min=min_val, max=max_val, step=1, value=val,
                              description=k, layout=Layout(width='auto'))
  note_default[k] = val

for n in range(n_notes):
    
    low_slider = note_sliders[note_keys[(n - 1) % n_notes]]
    curr_slider = note_sliders[note_keys[n]]
    high_slider = note_sliders[note_keys[(n + 1) % n_notes]]
        
    
    slider_neighbours[curr_slider] = {'low': low_slider,
                                      'high': high_slider}

    
cent_margin = 5

def clip_value(change):
    owner = change["owner"]
    neighbours = slider_neighbours[owner]
    
    low_value = neighbours['low'].value + cent_margin
    high_value = neighbours['high'].value - cent_margin
    
    if owner is note_sliders[note_keys[0]]:
        low_value -= octave_cents
    elif owner is note_sliders[note_keys[-1]]:
        high_value += octave_cents
    
    if owner.value < low_value:
        owner.value = low_value
    elif owner.value > high_value:
        owner.value = high_value
        
        
for slider in note_sliders.values():
    slider.observe(clip_value, 'value')
    

def notes2theta(notes_dict):
    vals = [octave_fit(notes_dict[key]) for key in fifth_keys]
    theta = np.array(vals) * 2 * np.pi / octave_cents
    
    return theta


default_theta = notes2theta(note_default)

plt.ioff()
plt.close(2)

fig = plt.figure(2)
fig.canvas.header_visible = False

ax = fig.add_subplot(1, 1, 1, projection='polar')
polar, = ax.plot(default_theta, r, marker="o", mfc="k", mec="k")

ax.set_yticks([1])
ax.set_yticklabels([])
ax.set_ylim(0, 1.1)
ax.set_xticks(default_theta[:-1])
ax.set_xticklabels(fifth_keys[:-1])
plt.grid(False, axis='x')

ax.set_theta_direction(-1)
ax.set_theta_zero_location('N')

ax.spines['polar'].set_visible(False)

plt.tight_layout()

audio_out = Output(layout=Layout(width='auto'))
plot_out = fig.canvas

def f(**kwargs):
    #with plot_out:
    theta = notes2theta(kwargs)
    polar.set_data(theta, r)
    ax.set_xticks(theta[:-1])
    #    plot_out.clear_output(wait=True)
    #    display(ax.figure)
    fig.canvas.draw()
    fig.canvas.flush_events()
    

interactive_plot = interactive_output(f, note_sliders)
#output = interactive_plot.children[-1]
# output.layout.height = '350px'
# interactive_plot

# with plot_out:
#     plot_out.clear_output(wait=True)
#     display(ax.figure)

grid[:n_notes, 0] = plot_out

for n, k in enumerate(note_keys):
    grid[n, 1] = note_sliders[k]


#slider_ui = VBox([HBox([Label(k), note_sliders[k]]) for k in note_keys])
#notes_ui = VBox([slider_ui, fig.canvas])

sampling_rate = 44100      # Sample rate (Hz)
f_root = 261.626 # TODO Choose f_root


def play_notes(b, autoplay=True):
    freqs = [note_sliders[k].value for k in note_keys]
    freqs.append(freqs[0] + octave_cents)
    audio_data = synthesize(freqs, fref=f_root, fs=sampling_rate)
    audio_out.clear_output()
    with audio_out:
        display(Audio(audio_data, rate=sampling_rate, autoplay=autoplay))

button = Button(description="Listen to the scale", layout=Layout(width='auto'))

# audio_ui = HBox([button, audio_out])

# ui = VBox([notes_ui, audio_ui])

# display(ui, interactive_plot)

button.on_click(play_notes)
play_notes(button, autoplay=False)

grid[-1, 0] = audio_out
grid[-1, 1] = button


display(grid)

GridspecLayout(children=(Canvas(header_visible=False, layout=Layout(grid_area='widget001'), toolbar=Toolbar(to…